<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Transfer-result" data-toc-modified-id="Transfer-result-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Transfer result</a></span><ul class="toc-item"><li><span><a href="#Get-the-label-file-content" data-toc-modified-id="Get-the-label-file-content-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Get the label file content</a></span></li></ul></li></ul></div>

## Transfer result
Transfer the label result to a excel file

In [33]:
import datetime
import pandas as pd
import numpy as np

In [56]:
def transfer_time(date_time, seconds, date_time_format='%d:%H:%M:%S'):
    """
    Add seconds to the date time and transfer to the target format
    
    Parameters
    ----------
    date_time : datetime object
        The date time we want to start with, here is the reset acquisition time
    seconds : int
        Seconds going to add to the date_time
    date_time_format : str
        Final format of date_time. Defaults is '%d:%M:%H:%S'
        
    Returns
    -------
    target_time : str
        Final date time in string format
        
    Examples
    --------
    Add seconds to the datetime
    
    >>> import datetime
    >>> original_time = datetime.datetime(2024, 1, 30, 10, 50, 0)
    >>> seconds = 40
    >>> format_ = '%d-%M:%H:%S'
    >>> transfer_time(original_time, seconds, format_)
    '30-10:50:40'
    """
    
    temp_time = date_time + datetime.timedelta(seconds=seconds)
    return temp_time.strftime(format=date_time_format)

def insert_row(df, idx, row):
    """
    Insert a row to a dataframe in a specific position
    
    Parameters
    ----------
    df : pandas.DataFrame
        dataframe for operation
    idx : int
        index to insert the row, insert below the row
    row : series
        Row to insert
        
    Returns
    -------
    result_df : pandas.DataFrame
    """
    if isinstance(row, pd.Series):
        row = pd.DataFrame(row).T
    df = pd.concat([df[:idx], row, df[idx:]], axis=0).reset_index(drop = True)
    return df
    
    

### Get the label file content


In [158]:
label_file_path = './data/mouse2_label.txt'
label_file = open(label_file_path, 'r').read().split('\n')
print('\n'.join(label_file[:10]))

acquisition_time = datetime.datetime.strptime(
    ' '.join(label_file[3].split(' ')[2:]), '%Y-%m-%d %H:%M:%S')
sampling_rate = int(label_file[4].split(' ')[2])

READ ONLY! DO NOT EDIT!
4-INIT 3-Wake 2-REM 1-NREM
Save time: 2024-01-23 20:11:53
Acquisition time: 2024-01-14 07:00:10
Sampling rate: 305
==========Marker==========
==========Start-End==========
==========Sleep stage==========
14:07:00:10, 0, 1, 14:07:10:19, 609, 0, 3, Wake
14:07:10:20, 610, 1, 14:07:10:32, 622, 0, 1, NREM


In [159]:
# Get the sleep stage dataframe
columns = ['start_time', 'start_time_sec', 'start_code',
           'end_time', 'end_time_sec', 'end_code',
           'state_code', 'state']
sleep_stage = label_file[label_file.index('==========Sleep stage==========')+1: ]
df = pd.DataFrame(data=sleep_stage, columns=['string'])
df = df['string'].str.split(', ', expand=True)
df.columns = columns

In [160]:
# Transfer the start and end time to datetime format
df['start_time'] = df['start_time_sec'].apply(
    lambda x: transfer_time(acquisition_time, int(x), '%Y-%m-%d %H:%M:%S'))

df['end_time'] = df['end_time_sec'].apply(
    lambda x: transfer_time(acquisition_time, int(x), '%Y-%m-%d %H:%M:%S'))

# Change the data type
df['start_time_sec'] = df['start_time_sec'].astype(int)
df['end_time_sec'] = df['end_time_sec'].astype(int)

lst = df['start_time_sec'].copy()
lst[0] = 1
df['start_time_sec'] = lst

df.head()

,start_time,start_time_sec,start_code,end_time,end_time_sec,end_code,state_code,state
0,2024-01-14 07:00:10,1,1,2024-01-14 07:10:19,609,0,3,Wake
1,2024-01-14 07:10:20,610,1,2024-01-14 07:10:32,622,0,1,NREM
2,2024-01-14 07:10:33,623,1,2024-01-14 07:13:49,819,0,3,Wake
3,2024-01-14 07:13:50,820,1,2024-01-14 07:14:04,834,0,1,NREM
4,2024-01-14 07:14:05,835,1,2024-01-14 07:14:19,849,0,3,Wake


In [161]:
# Only 2 possible situations, one is end_time_sec % 3600 == 0
# The other one is int(end_time_sec/3600) > int(start_time_sec/3600)
new_df = pd.DataFrame(columns=columns)
for idx, row in df.iterrows():
    if row['end_time_sec']%3600 == 0:
        new_df = insert_row(new_df, idx, row)
        # Just add a row and nothing else
        new_row = pd.Series([
            row['end_time'], row['end_time_sec'], ' ',
            row['end_time'], row['end_time_sec'], '5',
            ' ', 'MARKER'
        ], index=columns)
        new_df = insert_row(new_df, new_df.shape[0], row)
        continue
        
    if int(row['end_time_sec']/3600) > int(row['start_time_sec']/3600):
        seconds_ = (int(row['start_time_sec']/3600)+1)*3600
        previous_row = pd.Series([
            row['start_time'], row['start_time_sec'], '1',
            transfer_time(acquisition_time, seconds_,'%Y-%m-%d %H:%M:%S'),
            seconds_, '0', row['state_code'], row['state']
        ], index=columns)
        
        new_row = pd.Series([
            transfer_time(acquisition_time, seconds_,'%Y-%m-%d %H:%M:%S'), 
            seconds_, ' ',
            transfer_time(acquisition_time, seconds_,'%Y-%m-%d %H:%M:%S'), 
            seconds_, ' ', '5', 'MARKER'
        ], index=columns)
        
        below_row = pd.Series([
            transfer_time(acquisition_time, seconds_+1,'%Y-%m-%d %H:%M:%S'),
            seconds_+1, '1', row['end_time'], row['end_time_sec'], 
            '0', row['state_code'], row['state']
        ], index=columns)
        
        
        new_df = insert_row(new_df, new_df.shape[0], previous_row)
        new_df = insert_row(new_df, new_df.shape[0], new_row)
        while int(below_row['end_time_sec']/3600) > int(below_row['start_time_sec']/3600):
            row = below_row
            seconds_ = (int(row['start_time_sec']/3600)+1)*3600
            previous_row = pd.Series([
                row['start_time'], row['start_time_sec'], '1',
                transfer_time(acquisition_time, seconds_,'%Y-%m-%d %H:%M:%S'),
                seconds_, '0', row['state_code'], row['state']
            ], index=columns)

            new_row = pd.Series([
                transfer_time(acquisition_time, seconds_,'%Y-%m-%d %H:%M:%S'), 
                seconds_, ' ',
                transfer_time(acquisition_time, seconds_,'%Y-%m-%d %H:%M:%S'), 
                seconds_, ' ', '5', 'MARKER'
            ], index=columns)

            below_row = pd.Series([
                transfer_time(acquisition_time, seconds_+1,'%Y-%m-%d %H:%M:%S'),
                seconds_+1, '1', row['end_time'], row['end_time_sec'], 
                '0', row['state_code'], row['state']
            ], index=columns)
            new_df = insert_row(new_df, new_df.shape[0], previous_row)
            new_df = insert_row(new_df, new_df.shape[0], new_row)
            
        new_df = insert_row(new_df, new_df.shape[0], below_row)
        continue
    
    new_df = insert_row(new_df, new_df.shape[0], row)
    
df = new_df
del new_df

df['bout_duration'] = df.apply(
    lambda x: x[4]-x[1] + 1 if x[7] !='MARKER' else '', axis=1)

In [97]:
df.to_csv('./data/test.csv')

In [163]:
# Analysis for each hour
df['hour'] = df['start_time_sec'].apply(lambda x: int(x/3600) if x%3600!=0 else '')
analyse_df = pd.DataFrame()

analyse_df['date_time'] = [df['start_time'][0]] + list(df[df['state']=='MARKER']['start_time'])

features = []
for each in df[df['state'] != 'MARKER'].groupby('hour'):
    df_ = each[1]
    temp_lst = []
    for phase in ["NREM", "REM", "Wake", "INIT"]:
        _duration = df_[df_["state"] == phase]["bout_duration"].sum()
        _bout = df_[df_["state"] == phase]["bout_duration"].count()
        temp_lst += [_duration, _bout, round(_duration / _bout, 2) 
                     if _bout != 0 else 0, round(_duration / 3600, 2)]
    features.append(temp_lst)
        
analyse_df[['NREM_duration', 'NREM_bout', "NREM_ave", "NREM_percentage", 
            'REM_duration', 'REM_bout', "REM_ave", "REM_percentage", 
            'WAKE_duration', 'WAKE_bout', "WAKE_ave", "WAKE_percentage", 
            'INIT_duration', 'INIT_bout', "INIT_ave", "INIT_percentage"]] = features

analyse_df[
    ['NREM_duration', 'NREM_bout', 'REM_duration', 'REM_bout', 'WAKE_duration', 
     'WAKE_bout', 'INIT_duration', 'INIT_bout']
] = analyse_df[
    ['NREM_duration', 'NREM_bout', 'REM_duration', 'REM_bout', 'WAKE_duration', 
     'WAKE_bout', 'INIT_duration', 'INIT_bout']].astype(int)

In [165]:
writer = pd.ExcelWriter('./data/result.xlsx', datetime_format='yyyy-mm-dd hh:mm:ss')
pd.concat([df, analyse_df], axis=1).to_excel(
    excel_writer=writer, sheet_name='All', index=False)

writer.close()